In [58]:
from pathlib import Path
from utils.get_input import get_input
import numpy as np


inp = Path("./test_input").read_text()
inp = get_input(16)
inp

'#############################################################################################################################################\n#.#.........#.#...#...#...........#.....#.......#...#...........#.............#.....#.......#.......................................#......E#\n#.#.#.#####.#.#.#.#.#.#.#######.#.#.###.###.#.###.#.#.###.#######.#######.###.#.###.#.###.#.#######.###.#.#.#.#.#.#.###.###########.#####.#.#\n#.#.#.....#...#.#...#.......#...#...#.......#.................#.....#.....#.#...#...#.#...#.#.......#...#.#...#...#.#...#...#...#.........#.#\n#.#.#####.#####.#.###.#####.#.#####.###.#######.#####.###.###.#.#####.#####.#####.#####.###.#.#######.###.#.#######.#.#.#.#.#.#.#.###########\n#.#.#...#...#...#.....#...#.#.........#.#.......#.#.....#.#...#.#.....#...........#.....#.....#...#.......#...#.#.....#.#.#.#.#.#...#.......#\n#.#.###.###.#.###.#.#.#.#.#####.###.#.#.#.#######.#.#.#.###.###.#.#####.#############.#.#######.###.#.#.###.#.#.#.#####.#.#.#.#.#####.#####.#

In [59]:
inp_np = np.array([list(i) for i in inp.split("\n")])
inp_np

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '#', ..., '.', 'E', '#'],
       ['#', '.', '#', ..., '#', '.', '#'],
       ...,
       ['#', '#', '#', ..., '#', '.', '#'],
       ['#', 'S', '.', ..., '.', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

In [60]:
all_dots = np.argwhere(inp_np == ".")
S = np.argwhere(inp_np == "S")
E = np.argwhere(inp_np == "E")

all_dots_s_e = np.concatenate([all_dots, S, E])
# all_dots_s_e

In [66]:
# from dijkstar import Graph, find_path
import rustworkx as rx


num_nodes = (len(all_dots_s_e) * 4) + 4

all_dots_s_e_map = {tuple(n): i * 4 for i, n in enumerate(all_dots_s_e)}

graph = rx.PyDiGraph()
graph.add_nodes_from(range(num_nodes))

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

for n, i in all_dots_s_e_map.items():
    y, x = n

    graph.add_edges_from([(i + LEFT, i + UP, 1000)])
    graph.add_edges_from([(i + RIGHT, i + UP, 1000)])
    under = (y + 1, x)
    if under in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[under] + UP, i + UP, 1)])

    graph.add_edges_from([(i + UP, i + RIGHT, 1000)])
    graph.add_edges_from([(i + DOWN, i + RIGHT, 1000)])
    left_of = (y, x - 1)
    if left_of in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[left_of] + RIGHT, i + RIGHT, 1)])

    graph.add_edges_from([(i + RIGHT, i + DOWN, 1000)])
    graph.add_edges_from([(i + LEFT, i + DOWN, 1000)])
    above = (y - 1, x)
    if above in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[above] + DOWN, i + DOWN, 1)])

    graph.add_edges_from([(i + DOWN, i + LEFT, 1000)])
    graph.add_edges_from([(i + UP, i + LEFT, 1000)])
    right_of = (y, x + 1)
    if right_of in all_dots_s_e_map:
        graph.add_edges_from([(all_dots_s_e_map[right_of] + LEFT, i + LEFT, 1)])

s_i = (len(all_dots_s_e) - 2) * 4
e_i = (len(all_dots_s_e) - 1) * 4


ps = rx.dijkstra_shortest_path_lengths(graph, s_i + RIGHT, edge_cost_fn=int)

c1 = ps[e_i + RIGHT]
c2 = ps[e_i + DOWN]
c3 = ps[e_i + LEFT]
c4 = ps[e_i + UP]

shortest = int(min(c1, c2, c3, c4))
print("Part 1:", shortest)

print(f"Shortest each dir: RIGHT: {c1}, DOWN: {c2}, LEFT: {c3}, UP: {c4}")

Part 1: 94444
Shortest each dir: RIGHT: 94444.0, DOWN: 95444.0, LEFT: 96444.0, UP: 95444.0


In [68]:
from tqdm import tqdm
import itertools


sum = 0
d_combs = list(itertools.product([UP, RIGHT, DOWN, LEFT], [RIGHT]))

for n, i in tqdm(list(all_dots_s_e_map.items())[:-2]):
    y, x = n
    for d1, d2 in d_combs:
        c1 = ps[i + d1]
        if c1 >= shortest:
            continue
        ps2 = rx.dijkstra_shortest_path_lengths(graph, i + d1, edge_cost_fn=int)
        c2 = ps2[e_i + d2]
        cost_total = int(c1 + c2)
        y, x = n
        # print(y, x, cost_total, d1, d2)
        if cost_total == shortest:
            sum += 1
            break
sum + 2

100%|██████████| 10120/10120 [03:06<00:00, 54.23it/s]


502